In [1]:
# imports
from astropy.io import fits
from scipy import interpolate as sin
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotting as myP
import numpy as np
import csv 
import match
import SED

In [2]:
# path variables 
path_cat = 'D:\IfA REU\Catalogs'
path_fig = 'D:\IfA REU\IfA-REU-Project\Figures'
path_csv = 'D:\IfA REU\IfA-REU-Project\Lists'

# Get Data

In [3]:
# read AGN IDs
agnIDs = myP.ReadFile(path_csv+'\ID_RedYelBlu_AGNselection.csv')
# split agnIDs into AGN colors 
red_ID = np.array(agnIDs[0], dtype=int)
yel_ID = np.array(agnIDs[1], dtype=int)
blu_ID = np.array(agnIDs[2], dtype=int)
# print info  
print('Red:\t', len(red_ID))
print('Yel:\t', len(yel_ID))
print('Blu:\t', len(blu_ID)) # number is different from Figures\COSMOS_IR-Xray_selection_ZOOM_legend.png because we are not filtering for IRAC colors

Cropped:  ['Row 1 = red IR selected AGN. Row 2 = yellow both IR-Xray AGN. Row 3 = blue X-ray selected AGN. Values are ID_COSMOS2015.']
Red:	 886
Yel:	 1529
Blu:	 647


In [4]:
# read redshifts
zAll = np.array(myP.ReadFile(path_cat+'\COSMOS_z_matches.csv'))
zID, zZ = zAll.T
# get correct data type
zID = np.array(zID, dtype=int)
zZ  = np.array(zZ,  dtype=float)
# apply mask to remove bad redshifts
mask_z = (zZ >= 0) & (zZ < 99)
zID = zID[mask_z]
zZ  = zZ[mask_z]
# print info
print('Number of sources w/ z:', len(zID))

Cropped:  ['ID', 'z']
Number of sources w/ z: 485793


In [10]:
# match redshift against AGN color IDs
key_z_red, key_red_z= match.match(zID, red_ID)
key_z_yel, key_yel_z= match.match(zID, yel_ID)
key_z_blu, key_blu_z= match.match(zID, blu_ID)

# get redshift arrays for AGN by color 
red_z = zZ[key_z_red]
yel_z = zZ[key_z_yel]
blu_z = zZ[key_z_blu]

# get AGN arrays w/ redshifts
red_ID_wZ = red_ID[key_red_z]
yel_ID_wZ = yel_ID[key_yel_z]
blu_ID_wZ = blu_ID[key_blu_z]

# verify 
print('Red:\t', len(red_ID_wZ), '=', len(red_z))
print('Yel:\t', len(yel_ID_wZ), '=', len(yel_z))
print('Blu:\t', len(blu_ID_wZ), '=', len(blu_z))


Red:	 886 = 886
Yel:	 1529 = 1529
Blu:	 647 = 647


In [6]:
# open COSMOS 2020 catalog file and get data 
inf20 = fits.open(path_cat+'\COSMOS2020_CLASSIC_R1_v2.0_master.fits')
data20 = inf20[1].data
inf20.close()

In [7]:
Fnu_uJy = SED.GetPhotometry(data20)
IDs_all = SED.GetID(data20)
lam_A   = SED.GetObservedWavelengths_A()

Array shape:	 (1720700, 20)
Array shape:	 (1720700,)
Array shape:	 (20,)


In [8]:
# match AGN color IDS against photometry 
key_IDall_red, key_red_IDall = match.match(IDs_all, red_ID_wZ)
key_IDall_yel, key_yel_IDall = match.match(IDs_all, yel_ID_wZ)
key_IDall_blu, key_blu_IDall = match.match(IDs_all, blu_ID_wZ)

# get photometry for AGN colors 
red_Fnu_uJy = Fnu_uJy[key_IDall_red]
yel_Fnu_uJy = Fnu_uJy[key_IDall_yel]
blu_Fnu_uJy = Fnu_uJy[key_IDall_blu]

# get AGN arrays with photometry 
red_ID_wZ_wPhot = red_ID_wZ[key_red_IDall]
yel_ID_wZ_wPhot = yel_ID_wZ[key_yel_IDall]
blu_ID_wZ_wPhot = blu_ID_wZ[key_blu_IDall]

# verify 
print('Red:\t', len(red_ID_wZ_wPhot), '=', len(red_Fnu_uJy))
print('Yel:\t', len(yel_ID_wZ_wPhot), '=', len(yel_Fnu_uJy))
print('Blu:\t', len(blu_ID_wZ_wPhot), '=', len(blu_Fnu_uJy))


Red:	 886 = 886
Yel:	 1529 = 1529
Blu:	 647 = 647


In [12]:
# get table of rest wavelengths for AGN colors
red_lamR_A = SED.ConvertToRestWavelength(red_z)
yel_lamR_A = SED.ConvertToRestWavelength(yel_z)
blu_lamR_A = SED.ConvertToRestWavelength(blu_z)

Array shape:	 (886, 20)
Array shape:	 (1529, 20)
Array shape:	 (647, 20)


In [13]:
# convert Fnu to lamFlam
red_lamFlam_ergscm2 = SED.ConvertToEnergyDensity(red_lamR_A, red_Fnu_uJy)
yel_lamFlam_ergscm2 = SED.ConvertToEnergyDensity(yel_lamR_A, yel_Fnu_uJy)
blu_lamFlam_ergscm2 = SED.ConvertToEnergyDensity(blu_lamR_A, blu_Fnu_uJy)

Array shape:	 (886, 20)
Array shape:	 (1529, 20)
Array shape:	 (647, 20)


In [14]:
# normalize each agn source
# agn_lamflux_lam_ergscm2_NORM = SED.NormalizeSED_1um(lamR_A, agn_lamflux_lam_ergscm2)
red_lamFlam_ergscm2_norm = SED.NormalizeSED_1um(red_lamR_A, red_lamFlam_ergscm2)
yel_lamFlam_ergscm2_norm = SED.NormalizeSED_1um(yel_lamR_A, yel_lamFlam_ergscm2)
blu_lamFlam_ergscm2_norm = SED.NormalizeSED_1um(blu_lamR_A, blu_lamFlam_ergscm2)

Array shape:	 (886, 20)
Array shape:	 (1529, 20)
Array shape:	 (647, 20)


In [15]:
# set standard plot format
myP.SetStyle()

In [18]:
# concert angstrom to micron
red_lamR_um = red_lamR_A * 1E-4
yel_lamR_um = yel_lamR_A * 1E-4
blu_lamR_um = blu_lamR_A * 1E-4

In [20]:
# plot SEDs
SED.PlotSED(red_lamR_um, red_lamFlam_ergscm2_norm, title='Red')
SED.PlotSED(yel_lamR_um, yel_lamFlam_ergscm2_norm, title='Yellow')
SED.PlotSED(blu_lamR_um, blu_lamFlam_ergscm2_norm, title='Blue')